<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# GitHub - Send contributor activity on slack
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/GitHub/GitHub_Reopen_issue.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/Open_in_Naas_Lab.svg"/></a><br><br><a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=&template=template-request.md&title=Tool+-+Action+of+the+notebook+">Template request</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=GitHub+-+Reopen+issue:+Error+short+description">Bug report</a> | <a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Naas/Naas_Start_data_product.ipynb" target="_parent">Generate Data Product</a>

**Tags:** #github #activity #update #api #snippet #operations #slack 

**Author:** [Benjamin Filly](https://www.linkedin.com/in/benjamin-filly-05427727a/)

**Description:** This notebook explains how to send the GitHub activity on Slack

**References:**
- [GitHub REST API Documentation](https://docs.github.com/en/rest/issues/issues?apiVersion=2022-11-28#update-an-issue)

## Input

### Import libraries

In [4]:
import os
import json
from datetime import datetime
from github import Github
import naas
import pandas as pd
import requests
from naas_drivers import slack
import re
from datetime import datetime, timedelta
# The function "pd.set_option('display.max_colwidth', None)" is used in pandas, a data manipulation library in Python, to set the maximum width for displaying column contents in a tabular format to be unlimited, allowing the full content of each column to be displayed without truncation.
pd.set_option('display.max_colwidth', None)

### Setup Variables
- `token`: [GitHub token](https://help.github.com/en/github/authenticating-to-github/creating-a-personal-access-token-for-the-command-line)
- `owner`: owner of the repository
- `repo`: name of the repository
- `issue_number`: number of the issue

In [3]:
# Input variables
github_token = naas.secret.get("GITHUB_TOKEN") or "GITHUB_TOKEN"
repo_name = "jupyter-naas/awesome-notebooks" #example
contributor_profile = "Benjifilly"
slack_bot_token = naas.secret.get("SLACK_BOT_TOKEN") or "SLACK_TOKEN"
slack_channel = "naas-templates-contributors"

## Model

### Get Closed PRs
- Filter on contributor
- Get files added on PR (GitHub_Get_files_added_on_pull_request.ipynb)

In [68]:
# Connect to the GitHub API
g = Github(github_token)

# Get the repository
repo = g.get_repo(repo_name)

# Get the closed PR
pull_requests = repo.get_pulls(state='closed', sort='updated', direction='desc')
print("✅ Pull Requests fetched:", pull_requests.totalCount)

# Iterate over the pull requests and filter by assignee
assigned_pull_requests = [pr for pr in pull_requests if pr.assignee and pr.assignee.login == contributor_profile]

# Print the closed PR
print(len(assigned_pull_requests))

✅ Pull Requests fetched: 676
26


In [69]:
assigned_pull_requests

[PullRequest(title="feat: GitHub - List branches", number=1955),
 PullRequest(title="feat: Request - Handling Errors and Exceptions", number=1886),
 PullRequest(title="feat: convert multiple units", number=1960),
 PullRequest(title="feat: Slack - Send blocks to channel", number=1952),
 PullRequest(title="feat: Python - Unit converter", number=1943),
 PullRequest(title="feat: Python - Currency converter", number=1938),
 PullRequest(title="feat: Python - Pseudonym generator", number=1936),
 PullRequest(title="feat: Python - Get a random word", number=1931),
 PullRequest(title="feat: Python - Manage exception with try except", number=1940),
 PullRequest(title="feat: Pandas - Sort values by multiples columns", number=1900),
 PullRequest(title="feat: Request - Sending POST Requests with Data", number=1884),
 PullRequest(title="feat: Request - Basic HTTP GET Request", number=1882),
 PullRequest(title="Update and rename Python_Find_differences_between_stings.ipynb to Pyt…", number=1933),
 Pul

In [72]:
# for pr in assigned_pull_requests:
#     print(pr.number, pr.title, pr.merged_at)
    
#     # Get files added
    
#     # Prep data
#     tmp = {
#         "file_path": file_path,
#         "file_url": file_url,
#         "file_created": pr.merged_at,
#         "pr_number": pr.number,
#         "pr_title": pr.title,
#         "pr_url": pr.url,
#     }

1955 feat: GitHub - List branches 2023-06-26 07:48:58
1886 feat: Request - Handling Errors and Exceptions 2023-06-26 07:03:00
1960 feat: convert multiple units 2023-06-26 06:48:21
1952 feat: Slack - Send blocks to channel 2023-06-22 14:20:35
1943 feat: Python - Unit converter 2023-06-19 16:18:13
1938 feat: Python - Currency converter 2023-06-19 16:07:39
1936 feat: Python - Pseudonym generator 2023-06-19 16:04:35
1931 feat: Python - Get a random word 2023-06-19 13:25:27
1940 feat: Python - Manage exception with try except 2023-06-16 14:36:28
1900 feat: Pandas - Sort values by multiples columns 2023-06-16 07:01:09
1884 feat: Request - Sending POST Requests with Data 2023-06-16 06:47:03
1882 feat: Request - Basic HTTP GET Request 2023-06-16 06:33:29
1933 Update and rename Python_Find_differences_between_stings.ipynb to Pyt… 2023-06-15 09:16:49
1929 feat: Deepl - Translate text to .txt 2023-06-14 15:56:36
1903 feat: Pandas - Get n largest 2023-06-14 15:44:59
1905 feat: Pandas - Get n small

In [59]:
import requests
import naas

token = naas.secret.get("GITHUB_TOKEN") or "GITHUB_TOKEN"
owner = "jupyter-naas"
repo = "awesome-notebooks"
contributor = "BenjiFilly"
file_status = "added"

def get_closed_pull_requests(owner, repo):
    url = f"https://api.github.com/repos/{owner}/{repo}/pulls"
    params = {
        "state": "closed",
        "per_page": 100  # Adjust the number based on your requirements
    }
    headers = {
        "Authorization": f"Bearer {token}"
    }
    response = requests.get(url, params=params, headers=headers)
    pull_requests = response.json()
    return pull_requests

def get_files_added_on_pull_request(owner, repo, pull_number, file_status):
    files = []
    url = f"https://api.github.com/repos/{owner}/{repo}/pulls/{pull_number}/files"
    headers = {
        "Authorization": f"Bearer {token}"
    }
    response = requests.get(url, headers=headers)
    files_changed = response.json()
    for file in files_changed:
        filename = file["filename"]
        status = file["status"]
        if status == file_status:
            files.append(filename)
    return files

closed_pull_requests = get_closed_pull_requests(owner, repo)

for pull_request in closed_pull_requests:
    contributor_login = pull_request["user"]["login"]
    if contributor_login == contributor:
        pull_number = pull_request["number"]
        files_added = get_files_added_on_pull_request(owner, repo, pull_number, file_status)
        files_added
        break
        if len(files_added) > 0:
            for file in files_added:
                print("File added:", file)

### Get PRs

In [6]:
# Connect to the GitHub API
g = Github(github_token)
# Get the repository
repo = g.get_repo(repo_name)
# Get the closed PR
pull_requests = repo.get_pulls(state="open")

# Print the closed PR
print("✅ Pull Requests fetched:", pull_requests.totalCount)
data = []
for index, pr in enumerate(pull_requests):
    # Init
    assignee_login = None
    assignee = pr.raw_data.get("assignee")
    if assignee:
        assignee_login = assignee.get("login")
    tmp = {
        "title": pr.title,
        "number": pr.number,
        "url": pr.raw_data.get("url"),
        "assignee": assignee_login,
        "created_at": pr.raw_data.get("created_at"),
        "updated_at": pr.raw_data.get("updated_at"),
    }
    data.append(tmp)

df_pr = pd.DataFrame(data)
df_pr.head(3)

✅ Pull Requests fetched: 23


,title,number,url,assignee,created_at,updated_at
0,feat: GitHub send contributor activity on slack,1961,https://api.github.com/repos/jupyter-naas/awesome-notebooks/pulls/1961,Benjifilly,2023-06-23T10:19:01Z,2023-06-27T09:01:22Z
1,feat: Naas - Create templates using MyChatGPT,1947,https://api.github.com/repos/jupyter-naas/awesome-notebooks/pulls/1947,FlorentLvr,2023-06-16T07:36:06Z,2023-06-20T09:21:59Z
2,feat: LangChain - Create embeddings from awesome-notebooks repository,1942,https://api.github.com/repos/jupyter-naas/awesome-notebooks/pulls/1942,Dr0p42,2023-06-15T14:51:14Z,2023-06-15T14:51:16Z


### Filter dataframe on contributor
Do not forget to use .reset_index(drop=True)

In [7]:
contributor_filtered_df = df_pr[df_pr['assignee'] == contributor_profile].reset_index(drop=True)
print("✅ PR Opened:", len(contributor_filtered_df))
contributor_filtered_df.head(1)

✅ PR Opened: 3


,title,number,url,assignee,created_at,updated_at
0,feat: GitHub send contributor activity on slack,1961,https://api.github.com/repos/jupyter-naas/awesome-notebooks/pulls/1961,Benjifilly,2023-06-23T10:19:01Z,2023-06-27T09:01:22Z


### Get PR in Review and WIP

Find if "https://github.com/jupyter-naas/awesome-notebooks/issues/" in comments or PR description

In [45]:
# Init - Create empty list
prs_in_review = []
prs_work_in_progress = []
prs_linked_to_issues = []
prs_numbers = []

# Loop in df and last message on each PR
for row in contributor_filtered_df.itertuples():
    # Display PR number
    pr_number = row.number
    
    # Append prs numbers
    prs_numbers.append(str(pr_number))
    
    # Get PR object
    pr = next((pr for pr in pull_requests if pr.number == pr_number), None)
    
    # If PR exists then get comments
    if pr:
        # Get PR description
        pr_description = pr.body
        issue_number = None
        if pr_description:
            # Split description to get issue number
            split_result = pr_description.split("https://github.com/jupyter-naas/awesome-notebooks/issues/")

            # If list > 1 then issue number exists
            if len(split_result) > 1:
                # Get issue number
                issue_number = split_result[1]
        if issue_number:
            prs_linked_to_issues.append(issue_number)

        # Get comments from PR
        comments = pr.get_issue_comments()
        
        # Check if nb comments > 0 else PR status = WIP
        if comments.totalCount > 0:
            last_comment = comments[comments.totalCount-1].body.lower()
            
            # Check if "read to review" else PR status = WIP
            if "ready to review" in last_comment:
                prs_in_review.append(pr)
            else:
                prs_work_in_progress.append(pr)
        else:
            prs_work_in_progress.append(pr)
            
print("✅ Pull Requests in Review:", len(prs_in_review))
print("👨‍💻 Pull Requests Work in Progress:", len(prs_work_in_progress))
print("💻 Issues linked in PRs:", len(prs_linked_to_issues))

✅ Pull Requests in Review: 1
👨‍💻 Pull Requests Work in Progress: 2
💻 Issues linked in PRs: 2


### Get issues by contributor

In [32]:
# Get the repository
repository = g.get_repo(f'{repo_name}')

# Get issues assigned to the contributor
issues = repository.get_issues(assignee=contributor_profile)

# Print number of issues
print("✅ Issues fetched:", issues.totalCount)

✅ Issues fetched: 7


In [51]:
# Init variables
new_issues = []

# Loop
for issue in issues:
    # Init
    issue_number = issue.number
    
    # List events in issues
    events = issue.get_events()
    list_events = [event.event for event in events]
    
    # Exclude issues linked to PR or with events connected
    if (str(issue_number) not in prs_linked_to_issues) and ('connected' not in list_events) and (str(issue_number) not in prs_numbers):
        new_issues.append(issue)

print("✅ New issues:", len(new_issues))
new_issues

✅ New issues: 1


[Issue(title="Pandas - Concatenate dataframe fix", number=1878)]

### Create Slack block

In [54]:
prs_in_review.sort(key=lambda pr: pr.number, reverse=True)
prs_work_in_progress.sort(key=lambda pr: pr.number, reverse=True)
new_issues.sort(key=lambda issue: issue.number, reverse=True)
current_date = datetime.now().strftime("%B %d, %Y %H:%M")

blocks = [
    {
        "type": "header",
        "text": {
            "type": "plain_text",
            "text": f"{contributor_profile} - Activity update as of {current_date}",
            "emoji": True
        }
    },
    {
        "type": "divider"
    },
    {
        "type": "section",
        "text": {
            "type": "mrkdwn",
            "text": f"✅ *List of PR to be validated ({len(prs_in_review)}):*"
        }
    },
    *[
        {
            "type": "section",
            "text": {
                "type": "mrkdwn",
                "text": f"- <{pr.html_url}|#{pr.number} {pr.title}>"
            }
        }
        for pr in prs_in_review
    ],
    {
        "type": "divider"
    },
    {
        "type": "section",
        "text": {
            "type": "mrkdwn",
            "text": f"👨‍💻 *List of current PR ({len(prs_work_in_progress)}):*"
        }
    },
    *[
        {
            "type": "section",
            "text": {
                "type": "mrkdwn",
                "text": f"- <{pr.html_url}|#{pr.number} {pr.title}>"
            }
        }
        for pr in prs_work_in_progress
    ],
    {
        "type": "divider"
    },
    {
        "type": "section",
        "text": {
            "type": "mrkdwn",
            "text": f":bulb: *List of new issues ({len(new_issues)}):*"
        }
    },
    *[
        {
            "type": "section",
            "text": {
                "type": "mrkdwn",
                "text": f"- <{issue.html_url}|#{issue.number} {issue.title}>"
            }
        }
        for issue in new_issues
    ]
]

## Output

### Send message on Slack

In [55]:
slack.connect(slack_bot_token).send(slack_channel, text=None, blocks=blocks)

✉️ Message sent
